# DATA EDA

### Imports

In [1]:
#Imports
import pandas as pd
import numpy as np

### Dataframes Imports 

In [2]:
#Importing Transactions data
transactions_df= pd.read_csv("/Users/andretomaz/code/XavierLooyens/Churn_prediction/raw_data/transactions_v2.csv")
#Importing user logs data
user_logs_df = pd.read_csv("/Users/andretomaz/code/XavierLooyens/Churn_prediction/raw_data/user_logs_v2.csv")
#importing members data
members_df = pd.read_csv("/Users/andretomaz/code/XavierLooyens/Churn_prediction/raw_data/members_v3.csv")
# import training dataset
train_df = pd.read_csv("/Users/andretomaz/code/XavierLooyens/Churn_prediction/raw_data/train_v2.csv")

### Transactions Data

In [3]:
###### converting transaction_date and membership_date to datetime object
transactions_df['transaction_date'] = pd.to_datetime(transactions_df['transaction_date'], format='%Y%m%d')
transactions_df['membership_expire_date'] = pd.to_datetime(transactions_df['membership_expire_date'], format='%Y%m%d')

# removing duplicates and leaving only the latest transaction date
transactions_df_lt = transactions_df.loc[transactions_df.groupby('msno').transaction_date.idxmax()]

# creating remaining plan duration column
transactions_df_lt['remaining_plan_duration'] = transactions_df_lt['membership_expire_date'] - transactions_df_lt['transaction_date']

# creating discount column
transactions_df_lt['is_discount'] = transactions_df_lt.apply(lambda x: '0' if (x['actual_amount_paid'] -x['plan_list_price'])>=0 else '1', axis=1)

### User logs Data

In [4]:
# Converting date into datetime object
user_logs_df['date'] = pd.to_datetime(user_logs_df['date'], format='%Y%m%d')

#create new dataframe from transactions table with only msno and latest transaction date
latest_transactions_per_msno = transactions_df_lt[['msno', 'transaction_date']]

#Merge user logs with latest transaction date
merged_df = user_logs_df.merge(latest_transactions_per_msno, on='msno', how='left')

# drop msno's where transaction date is NaN
merged_df = merged_df.dropna(subset=['transaction_date'])

#removing rows where the user log data is before the last transaction date
user_logs_atd= merged_df.loc[merged_df['date']>=merged_df['transaction_date']]

# removing data column
user_logs_atd = user_logs_atd.drop(columns=['date','transaction_date'])

#groupby msno and summing all values
user_logs_atd = user_logs_atd.groupby('msno').sum()

### Merging

In [5]:
# Merging transactions, user logs and members data to Train dataframe
train_df = train_df.merge(transactions_df_lt, on='msno', how='left')
train_df = train_df.merge(user_logs_atd, on='msno', how='left')
train_df = train_df.merge(members_df, on='msno', how='left')

### Feature engineering

In [6]:
# Drop Gender column
train_df = train_df.drop(['gender'],axis=1)

In [7]:
# Drop NaN
train_df = train_df.dropna()

In [8]:
# Changing bd outliers to NaN
train_df['bd'] = train_df['bd'].apply(lambda x: np.nan if x <14 or x > 75 else x)

In [9]:
# Converting remaining plan duration to int
train_df['remaining_plan_duration'] = train_df['remaining_plan_duration'].dt.days

In [10]:
# Converting total seconds to hours
train_df['total_secs']= round(train_df['total_secs']/3600,2)

In [11]:
# Creating average usage (hours) per day from latest transaction
train_df['usage_from_ltd'] = round(train_df['total_secs']/train_df['remaining_plan_duration'],2)
# replacing Nan with 0 
train_df['usage_from_ltd'].replace([np.nan], 0, inplace=True)

In [12]:
# converting registration time to datetime object
train_df['registration_init_time'] = pd.to_datetime(train_df['registration_init_time'], format='%Y%m%d')

In [13]:
# creating discount percentage column
train_df['discount_percentage'] = round((train_df['plan_list_price'] - train_df['actual_amount_paid'])/train_df['plan_list_price'],2)

train_df['discount_percentage'].replace([np.nan,-0.01,0.01], 0, inplace=True)

### Cyclic Encoding

In [15]:
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [16]:
# cyclic encoding transaction date
#year
train_df['last_transaction_year'] = train_df['transaction_date'].dt.year

#month
train_df['last_transaction_month'] = train_df['transaction_date'].dt.month
train_df = encode(train_df, 'last_transaction_month', 12)

#day
train_df['last_transaction_day'] = train_df['transaction_date'].dt.day
train_df= encode(train_df, 'last_transaction_day', 31)

train_df =train_df.drop(['last_transaction_month', 'last_transaction_day'],axis=1)

In [17]:
# cyclic encoding membership expire date
#year
train_df['expire_year'] = train_df['membership_expire_date'].dt.year

#month
train_df['expire_month'] = train_df['membership_expire_date'].dt.month
train_df = encode(train_df, 'expire_month', 12)

#day
train_df['expire_day'] = train_df['membership_expire_date'].dt.day
train_df= encode(train_df, 'expire_day', 31)

# droping month and day columns
train_df =train_df.drop(['expire_month', 'expire_day'],axis=1)

In [18]:
# cyclic encoding registration date
#year
train_df['registration_year'] = train_df['registration_init_time'].dt.year

#month
train_df['registration_month'] = train_df['registration_init_time'].dt.month
train_df = encode(train_df, 'registration_month', 12)

#day
train_df['registration_day'] = train_df['registration_init_time'].dt.day
train_df= encode(train_df, 'registration_day', 31)

# droping month and day columns
train_df =train_df.drop(['registration_month', 'registration_day'],axis=1)


In [19]:
# droping date columns to avoid data leakege
train_df =train_df.drop(['registration_init_time', 'membership_expire_date', 'transaction_date'],axis=1)

In [22]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 636730 entries, 1 to 970956
Data columns (total 37 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   msno                        636730 non-null  object 
 1   is_churn                    636730 non-null  int64  
 2   payment_method_id           636730 non-null  float64
 3   payment_plan_days           636730 non-null  float64
 4   plan_list_price             636730 non-null  float64
 5   actual_amount_paid          636730 non-null  float64
 6   is_auto_renew               636730 non-null  float64
 7   is_cancel                   636730 non-null  float64
 8   remaining_plan_duration     636730 non-null  int64  
 9   is_discount                 636730 non-null  object 
 10  num_25                      636730 non-null  float64
 11  num_50                      636730 non-null  float64
 12  num_75                      636730 non-null  float64
 13  num_985            

In [21]:
#train_df.to_csv('train_df_2.csv')